In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms,models
from torchsummary import summary
from datetime import datetime
from tqdm import tqdm
import sys
import matplotlib.pyplot as plt

In [2]:
print(sys.version_info)
print(torch.__version__)
print(torchvision.__version__)
print(torch.cuda.is_available())

sys.version_info(major=3, minor=9, micro=2, releaselevel='final', serial=0)
2.4.0+cpu
0.19.0+cpu
False


## RESNET

In [3]:
# train(pretrained=False)
model = models.resnet18(pretrained=False, num_classes=10)

C:\Users\ZXF\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\ZXF\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [4]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
# Fine-tune (pretrained=True) =>  can not use (num_classes=10) 
# modify the out_features in fc
model = models.resnet18(pretrained=True, )
# get the in_features of model
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 10)

C:\Users\ZXF\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:
summary(model, input_size=(3, 224, 224), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

## parameters

In [8]:
classes = ('plane', 'car' , 'bird',
    'cat', 'deer', 'dog',
    'frog', 'horse', 'ship', 'truck')

In [9]:
# dataset
# input_shape = 32
num_classes = 10 # amount of categorize

#hyper
batch_size = 64
num_epochs= 5
learning_rate = 1e-3

# gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
device

device(type='cpu')

In [11]:
def get_mean_and_std(dataset):
    '''Compute the mean and std value of dataset.'''
    '''dataset: 0-1 range (ToTensor())'''
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True, num_workers=2)
    mean = torch.zeros(3)
    std = torch.zeros(3)
    print('==> Computing mean and std..')
    for inputs, targets in tqdm(dataloader):
        for i in range(3):
            mean[i] += inputs[:, i, :, :].mean()
            std[i] += inputs[:, i, :, :].std()
    mean.div_(len(dataset))
    std.div_(len(dataset))
    return mean, std

## dataset 与 dataloader

In [12]:
# offline calculate mean/std of training data
train_dataset = torchvision.datasets.CIFAR10(root = 'data',
                                           train = True,
                                           download = True,
                                           transform = transforms.ToTensor())
test_dataset = torchvision.datasets.CIFAR10(root = 'data',
                                           train = False,
                                           download = True,
                                           transform = transforms.ToTensor())

Files already downloaded and verified
Files already downloaded and verified


In [13]:
get_mean_and_std(train_dataset)

==> Computing mean and std..


100%|███████████████████████████████████████████████████████████████████████████| 50000/50000 [01:51<00:00, 450.16it/s]


(tensor([0.4914, 0.4822, 0.4465]), tensor([0.2023, 0.1994, 0.2010]))

In [14]:
transform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))
])

In [15]:
train_dataset = torchvision.datasets.CIFAR10(root = 'data',
                                           train = True,
                                           download = True,
                                           transform = transform)
test_dataset = torchvision.datasets.CIFAR10(root = 'data',
                                           train = False,
                                           download = True,
                                           transform = transform)

Files already downloaded and verified
Files already downloaded and verified


In [16]:
# from torch.utils.data import Dataset, DataLoader
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = False)

In [17]:
# get one batch of data
images, labels = next(iter(train_loader))

In [18]:
train_dataset.data.shape

(50000, 32, 32, 3)

In [19]:
test_dataset.data.shape

(10000, 32, 32, 3)

In [20]:
images.shape

torch.Size([64, 3, 224, 224])

In [21]:
len(train_loader)

782

## model arch

In [22]:
model = models.resnet18(pretrained=True,)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 10)
model = model.to(device)

## model train

In [23]:
criterion = nn.CrossEntropyLoss()
# optimzier = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum=0.9,weight_decay=5e-4)
total_batch = len(train_loader)

In [ ]:
for epoch in range(num_epochs):
    # get images and labels from each epoch
    for batch_idx, (images, labels) in enumerate(train_loader):
        # feed the device by images and labels and return a tensor
        images = images.to(device)
        labels = labels.to(device)
        
        # ---Forward---#
        # put the tensor you got into model,return the prediction(out)
        out = model(images)
        # calculate loss
        loss = criterion(out, labels)
        
        # 标准的处理，用 validate data；这个过程是监督训练过程，用于 early stop
        n_corrects = (out.argmax(axis=1) == labels).sum().item()
        acc = n_corrects/labels.size(0)
        
        # ---Backward---#
        # set the grads to None
        optimizer.zero_grad()
        loss.backward()
        # update parameters
        optimizer.step()
        
        #Obviouse the grad
        if (batch_idx+1) % 100 == 0:
            print(f'{datetime.now()},{epoch+1}/{num_epochs}, {batch_idx+1}/{total_batch}: {loss.item():.4f}')     

## model evaluation

In [ ]:
class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        Returns:
            Tensor: Normalized image.
        """
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
            # The normalize code -> t.sub_(m).div_(s)

In [ ]:
unnormalize = UnNormalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))

In [49]:
total = 0
correct = 0

all_mis_preds = []
all_mis_labels = []
all_mis_images = []


for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    out = model(images)
    preds = torch.argmax(out, dim=1)
    # total= 64+64+64.....+64+16(the remains)
    total += images.size(0)
    correct += (preds == labels).sum().item()
    # preds == labels =>1
    # preds != labels =>0
    
    mis_preds_indice = torch.flatten((preds != labels).nonzero())
    mis_preds = preds[mis_preds_indice]
    mis_labels = labels[mis_preds_indice]
    mis_images = images[mis_preds_indice]
    
    all_mis_preds.extend(mis_preds)
    all_mis_labels.extend(mis_labels)
    
    for i in range(mis_images.size(0)):
        all_mis_images.append(unnormalize(mis_images[i]))
        
print(f'{correct}/{total}={correct/total}')

7528/10000=0.7528


In [ ]:
fig = plt.figure(figsize=(12, 12))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.tight_layout()
    plt.imshow(np.transpose(all_mis_images[i].cpu(), (1, 2, 0)), interpolation='none')
    plt.title("pred: {}, gt: {}".format(classes[all_mis_preds[i].item()], classes[all_mis_labels[i].item()]))
    plt.xticks([])
    plt.yticks([])

## save model

In [105]:
torch.save(model.state_dict(), 'resnet18_cifar10.ckpt')